# Chargement des fichiers ...

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from google.colab import files

print("Upload des fichiers 0-xxxx.csv, 1-xxxx.csv, ...")
uploaded = files.upload()

# On garde les données ainsi que l'association avec le libellé
all_X = []
all_y = []

NUM_SAMPLES = 50  # 50 points (x,y) donc 100 valeurs par geste

for fname in uploaded.keys():
    print("\nFichier :", fname)

    # extraire le label à partir du nom (ex: "0-xxxx.csv" -> 0)
    base = fname.split('-')[0]
    try:
        label = int(base)
    except ValueError:
        raise ValueError(f"Le fichier {fname} ne commence pas par un label numérique.")

    df = pd.read_csv(fname, header=None)

    for row in df.values:
        row = row.astype(np.float32) / 4095.0 # On normalise
        all_X.append(row)
        all_y.append(label)

# On converti en format Numpy
X = np.vstack(all_X)
y = np.array(all_y, dtype=np.int32)


Upload des fichiers 0-xxxx.csv, 1-xxxx.csv, ...


Saving 0-rond.csv to 0-rond.csv
Saving 1-autres.csv to 1-autres.csv

Fichier : 0-rond.csv

Fichier : 1-autres.csv


# On entraîne ...


In [4]:
# Mélanger et splitter train / val
N = X.shape[0]
idx = np.random.permutation(N)
X = X[idx]
y = y[idx]

split = int(0.8 * N)
X_train, X_val = X[:split], X[split:]
y_train, y_val = y[:split], y[split:]

num_classes = len(np.unique(y))

# Modèle MLP (Multi-Layer Perceptron) simple
model = tf.keras.Sequential([
    # La couche d'entrée : chaque geste est représenté par 100 valeurs (50 points X et 50 points Y).
    # On informe TensorFlow que chaque exemple aura exactement 100 features.
    tf.keras.layers.Input(shape=(NUM_SAMPLES * 2,)),

    # Première couche cachée : 16 neurones, activation ReLU.
    # ReLU permet d'apprendre des relations non linéaires (très important pour la classification).
    tf.keras.layers.Dense(16, activation="relu"),

    # Deuxième couche cachée : encore 16 neurones.
    # Elle combine les informations apprises dans la couche précédente.
    tf.keras.layers.Dense(16, activation="relu"),

    # Couche de sortie : un neurone par classe.
    # Softmax transforme les sorties en probabilités (ex: [0.9, 0.1] = classe 0).
    tf.keras.layers.Dense(num_classes, activation="softmax"),
])

model.compile(
    # Adam est un optimiseur qui ajuste automatiquement la vitesse d'apprentissage.
    # Il fonctionne bien pour la majorité des petits modèles.
    optimizer="adam",

    # La fonction de perte mesure l'erreur du modèle pendant l'entraînement.
    # Ici, sparse_categorical_crossentropy est utilisée quand les labels sont des entiers
    loss="sparse_categorical_crossentropy",

    # On demande à TensorFlow de calculer l'accuracy (le % de bonnes prédictions)
    # à chaque epoch (passage complet sur les données) sur le train et sur la validation.
    metrics=["accuracy"],
)

model.summary()

history = model.fit(
    # Données d'entraînement : les exemples (X) et leurs labels (y)
    X_train, y_train,

    # Données de validation : jamais utilisées pour apprendre,
    # servent uniquement à mesurer les performances du modèle
    # sur des exemples "neufs"
    validation_data=(X_val, y_val),

    # Nombre de fois que le modèle voit toutes les données d'entraînement.
    # 1 epoch = 1 passage complet sur le dataset.
    epochs=30,

    # Le modèle traite les données en petits groupes de 16 exemples.
    # Cela rend l'entraînement plus stable et plus rapide que tout traiter d'un coup.
    batch_size=16,
)

print("Dernière epoch :")
print("  train acc :", history.history["accuracy"][-1])
print("  val   acc :", history.history["val_accuracy"][-1])


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │         1,616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │            34 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,922 (7.51 KB)

 Trainable params: 1,922 (7.51 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 157ms/step - accuracy: 0.6719 - loss: 0.6573 - val_accuracy: 0.9167 - val_loss: 0.6011
Epoch 2/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.8177 - loss: 0.6203 - val_accuracy: 0.9167 - val_loss: 0.5672
Epoch 3/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.7969 - loss: 0.6044 - val_accuracy: 0.9167 - val_loss: 0.5366
Epoch 4/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.8594 - loss: 0.5803 - val_accuracy: 0.9167 - val_loss: 0.5095
Epoch 5/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.8698 - loss: 0.5420 - val_accuracy: 0.9167 - val_loss: 0.4826
Epoch 6/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.9089 - loss: 0.5239 - val_accuracy: 0.9167 - val_loss: 0.4622
Epoch 7/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.8880 - loss: 0.4965 - val_accuracy: 1.0000 - val_loss: 0.4411
Epoch 8/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.9531 - loss: 0.4588 - val_accuracy: 1.0000 - val_loss: 0.4144

# On exporte ...

In [5]:
# Conversion en TFLite float32
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Export en model_data.h
def write_c_array(tflite_bytes, var_name="model_data"):
    byte_str = ",".join(str(b) for b in tflite_bytes)
    return f"""
#include <stdint.h>

const unsigned char {var_name}[] = {{
{byte_str}
}};

const int {var_name}_len = {len(tflite_bytes)};
"""

c_code = write_c_array(tflite_model, "model_data")

with open("model_data.h", "w") as f:
    f.write(c_code)

print("Fichier model_data.h généré.")

files.download("model_data.h")

Saved artifact at '/tmp/tmpmvplkeyb'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 100), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  139241291057296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139241291058448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139241291057104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139241291055568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139241291059024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139241291055760: TensorSpec(shape=(), dtype=tf.resource, name=None)
Fichier model_data.h généré.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>